# Setup Jupyter notebook

In [1]:
from pathlib import Path
import sys

notebook_directory_parent = Path.cwd().resolve().parent.parent
print(notebook_directory_parent)
if str(notebook_directory_parent) not in sys.path:
    sys.path.append(str(notebook_directory_parent))

/cuBlackDream


# Setup to use Python libraries/modules

In [2]:
from CuISOX.DataWrangling.Kaggle.DigitRecognizer import ProcessDigitsData
from CuISOX.PyTorch.RecurrentNeuralNetworks.LSTM.Examples import LSTMWithLinearOutput
from CuISOX.utilities.configure_paths import DataPaths
from CuISOX.utilities.DataIO.KagglePaths import KagglePaths

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

In [3]:
data_paths = DataPaths()
kaggle_paths = KagglePaths()
kaggle_data_file_paths = kaggle_paths.get_all_data_file_paths()
digit_paths = kaggle_data_file_paths["DigitRecognizer"]

# Load Data
See [Long-Short Term Memory with Pytorch](https://www.kaggle.com/code/kanncaa1/long-short-term-memory-with-pytorch)

In [4]:
training_data_paths = DataPaths.get_path_with_substring(digit_paths, "train")
training_data_path = data_paths.Kaggle() / training_data_paths[0]
print(training_data_path)

/cuBlackDream/Data/Kaggle/DigitRecognizer/digit-recognizer/train.csv


In [5]:
process_digits_data = ProcessDigitsData()

process_digits_data.parse_csv(training_data_path)
process_digits_data.load_data()

## Model Parameters and Sizes, Configuration

Batch size, epoch, and iteration

Suppose $B \equiv$ batch size.

Take the total number of samples $N$ and divide by $B$ so to get "number of batches". Given $N_{\text{iters}} \equiv$ total number of iterations, with each iteration doing 1 batch, we can get the total number of epochs.

In [ ]:
batch_size = 100
n_iters = 6000
num_epochs = int( n_iters / (len(X_features_training) / batch_size))
print("Epoch Number: ", num_epochs)

In [ ]:
# Visualize one of the images in data set.
plt.imshow(X_features_numpy[42].reshape(28, 28))
plt.axis("off")
plt.title(str(y_targets_numpy[42]))
plt.show()

# Run Forward once

In [6]:
training_data_iterator = enumerate(process_digits_data.training_loader)

In [7]:
input_dim = 28
hidden_dim = 100
layer_dim = 1
output_dim = 10
sequence_length = 28
# EY: 20230906, we'll use one of the physical dimensions (I'm guessing width) as the "sequence" variable. So we
# imagine that each row is an input and each successive row makes a sequence of rows of pixels of the image.
lstm_with_linear_output = LSTMWithLinearOutput(input_dim, hidden_dim, layer_dim, output_dim, sequence_length)

In [8]:
i, (image_batch, batch_labels) = training_data_iterator.__next__()

In [9]:
example_output, example_loss = lstm_with_linear_output.run_on_image_batch(image_batch, batch_labels)

In [14]:
print(type(example_output))
print(type(example_loss))
print(example_output.size())
print(example_loss.size())
print(example_output.shape)
print(example_loss.shape)
print(example_output[0])
print(example_output[1])
print(example_loss.item())

<class 'torch.Tensor'>
<class 'torch.Tensor'>
torch.Size([100, 10])
torch.Size([])
torch.Size([100, 10])
torch.Size([])
tensor([ 0.0300, -0.1175, -0.1147,  0.0723, -0.0552, -0.0509, -0.0683, -0.0231,
        -0.0036, -0.0847], grad_fn=<SelectBackward0>)
tensor([ 0.0351, -0.1163, -0.1216,  0.0764, -0.0520, -0.0583, -0.0689, -0.0247,
        -0.0029, -0.0828], grad_fn=<SelectBackward0>)
2.2987165451049805
